In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit_aer
!pip install qiskit_machine_learning
!pip install qiskit_algorithms
!pip install matplotlib
!pip install pylatexenc
!pip install scipy
!pip install -U scikit-learn

!git clone https://github.com/IsaVia777/atelier_qml.git

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys 

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit_machine_learning.optimizers import COBYLA, SPSA
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, TwoLocal, ZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

SEED = 8398

In [ ]:
import sys
sys.path.insert(0, '/content/atelier_qml')

from utils import *

# Lab 4: Training a quantum classifier

**Objectives**
- Building a parametrized circuit 
- Angle embedding
- Variational quantum circuit (ansatz)  
- Classification of the Iris dataset


In [ ]:
x_train,y_train,x_test,y_test = get_iris(SEED)
nb_features = 4
nb_classes = 2

# Building a quantum classifier

## Step 1:  Parametrized quantum circuit

This parametrized quantum circuit is composed of two elements: 

* An embedding circuit 
* A circuit to learn the measurement basis

As always, the qubits are initialized in the zero state $|0\rangle$. The first circuit of a QML model will be the data embedding circuit. 

In [ ]:
# Data embedding circuit
x_params = [Parameter(f'x{str(i)}') for i in range(nb_features)]    
emb_circuit = angle_embedding(x_params,nb_features)

# Circuit to learn the measurement basis
ansatz = TwoLocal(nb_features, ['rz', 'rx'], 'cx', 'linear', reps=2, parameter_prefix='w')

qc = emb_circuit.compose(ansatz)
qc.draw('mpl')


In [ ]:
qc.decompose().draw('mpl')

## Circuit QNN

In [ ]:
# Random initialization of the weights
np.random.seed(SEED)
initial_weights = np.random.rand(ansatz.num_parameters) 

In [ ]:
# Interpretation function: parity 
# counts the nuumber of "1" in the bit string, x,
# and returns 0 is the number is even, 1 if it is odd
def parity(x):
    return '{:b}'.format(x).count('1') % 2

In [ ]:
# Initialize the optimizer
num_iter = 20
optimizer = COBYLA(maxiter=num_iter)

sampler_qnn = SamplerQNN(circuit=qc,  
                         input_params=emb_circuit.parameters,
                         weight_params=ansatz.parameters, 
                         interpret=parity, 
                         output_shape=nb_classes)

In [ ]:
# The probability to be classified as belonging to class 0 or 1 for a given datapoint,
# and a given parameter value , are obtained with the function `CircuitQNN.forward()`
probs = sampler_qnn.forward(x_train[0], initial_weights)
print(f">\n> Probability to belong in class 0: {probs[0][0]*100:.1f}%\n> Probability to belong in class 1: {probs[0][1]*100:.1f}%\n>")

## Training of the classifier with `Neural Network Classifier`

In [ ]:
# Instantiate a `NeuralNetworkClassifier` to train our model and make inferences
circuit_classifier = NeuralNetworkClassifier(neural_network=sampler_qnn,
                                             optimizer=optimizer,
                                             initial_point=initial_weights)
#Training the model
circuit_classifier.fit(x_train, y_train)

In [ ]:
# Score on the train set
train_acc = circuit_classifier.score(x_train, y_train)
test_acc = circuit_classifier.score(x_test, y_test)

print(f' > Train accuracy: {train_acc}\n > Test accuracy: {test_acc}')

# Exercise 4

Can we do better?<br>

Let's try using a more expressive feature map than angle embedding; for example, `ZZFeatureMap`.

In [ ]:
## Your code here
feature_map = None
##

# We are using the same circuit to learn the measurement basis
ansatz = TwoLocal(nb_features, ['rz', 'rx'], 'cx', 'linear', reps=2, parameter_prefix='w')

qc = feature_map.compose(ansatz)
qc.draw('mpl')

In [ ]:
## Your code here

circuit_qnn = SamplerQNN(circuit=qc,  
                         input_params=None,  ## Put the list of parameters here!
                         weight_params=ansatz.parameters, 
                         interpret=parity, 
                         output_shape=nb_classes)

In [ ]:
# Instanciate the class used to train the quantum classifier
circuit_classifier = NeuralNetworkClassifier(neural_network=circuit_qnn,
                                             optimizer=optimizer,
                                             initial_point=initial_weights)

# Train the model
circuit_classifier.fit(x_train, y_train)

In [ ]:
# Compute model accuracy on the training and test datasets
train_acc = circuit_classifier.score(x_train, y_train)
test_acc = circuit_classifier.score(x_test, y_test)

print(f">\n> Accuracy on the training set: {train_acc}\n> Accuracy on the test set: {test_acc}\n>")

You can also explore on your own, the effects of the following on your model's performance:
* different feature map or data embedding
* optimizer (number of iterations or switch optimizer for SPSA, for example)
* measurement interpretation function